In [ ]:
import pandas as pd
import numpy as np
bike=pd.read_csv("train.csv",encoding='utf-8')

In [ ]:
#날짜를 인덱스화 한다.
bike['일시']=bike['일시'].astype('str')
bike['일시']=pd.to_datetime(bike['일시'])
bike.set_index('일시',inplace=True)
bike['ds']=bike.index


In [ ]:
#그래프를 그릴 모듈과 prophet 모듈 import
from matplotlib import pyplot
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
#광동구 동대문구 그래프
plt.plot(bike['광동구'])
plt.plot(bike['동대문구'])
plt.show()

In [ ]:
#성동구 중랑구 그래프
plt.plot(bike['성동구'])
plt.plot(bike['중랑구'])
plt.show()

In [ ]:
#Prophet은 date열 이름이 ds 값의 열 이름이 y이어야 하므로 조건에 맞는 데이터 프레임을 만든다.
gwang2=pd.DataFrame({
    'ds' : bike['ds'],
    'y' : bike['광진구']
})
dong2 = pd.DataFrame({
    'ds' : bike['ds'],
    'y' : bike['동대문구']
})
sung2 = pd.DataFrame({
    'ds' : bike['ds'],
    'y' : bike['성동구']
})
jung2 = pd.DataFrame({
    'ds' : bike['ds'],
    'y' : bike['중랑구']
})


In [ ]:
#주말을 반영하기 위해서 holiday이면 명절 이름이 나오고 아니면 non-holiday가 나오는 holiday_df2생성
date_list2 = pd.date_range('2018-01-01', '2021-12-31')
kr_holidays2 = holidays.KR()
holiday_df2 = pd.DataFrame(columns=['ds','holiday'])
holiday_df2['ds'] = sorted(date_list2)
holiday_df2['holiday'] = holiday_df2.ds.apply(lambda x: kr_holidays2.get(x) if x in kr_holidays2 else 'non-holiday')

In [ ]:
#https://dacon.io/competitions/official/236029/codeshare/7311?page=1&dtype=recent를 참고했습니다
#그리드 서치를 위한 parameter 묶음을 만든다.
param_combined2 = [dict(zip(search_space2.keys(), v)) for v in itertools.product(*search_space2.values())]
mapes = []
for param in param_combined2:
   m=Prophet(**param)
   m.fit(gwang2)
#발생한 모든 조합을 cross_validation 진행하며
#initial: 2년(365*2일)을 학습 진행 후 period: 90일 단위로 horizon: 30일에 대한 평가를 진행한다.
   cv_df=cross_validation(m,initial='730 days', period='90 days', horizon='30 days', parallel='processes')
   df_p=performance_metrics(cv_df,rolling_window=1)
   mapes.append(df_p['mae'].values[0])

results_gwang2=pd.DataFrame(param_combined2)
results_gwang2['mapes']=mapes
Best_g=results_gwang.sort_values(by=['mapes'],axis=0).iloc[0][0:5]

In [ ]:
#상한과 하한을 지정해주면 하한 밑과 상한 위의 예측은 사라져서 성능 향상에 도움이 된다.
#make_future_dataframe(periods=365)으로 2021년 12월 31까지었던 것을 2022년 12월 31일 까지로 늘려준다. 
future = m.make_future_dataframe(periods=365)
#상한 20 하한 0
future['cap']=20
future['floor']=0

In [ ]:
#앞에서 구했던 best parameter값으로 모델을 구현한다.
#상한 20 하한 0을 설정했고 따라서 growth='logistic'을 추가했다.
#광진구부터 동대문구, 성동구, 중랑구 순으로 같은 방식으로 진행했다.
gwang2['cap']=20
gwang2['floor']=0
m_g2=Prophet(changepoint_prior_scale=1,seasonality_prior_scale=1,holidays_prior_scale=1,seasonality_mode='additive',growth='logistic')
m_g2.add_country_holidays(country_name='KR')
m_g2.fit(gwang2)
#모형으로 예측한 것을 그래프로 그렸다.
fore = m_g2.predict(future)
fig = m_g2.plot(fore)
fig